In [6]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=720)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 全撈 
one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
stock_df_raw = pd.read_sql(query_stock, engine)


In [26]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')


# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame

sub_category_list = pd.read_sql(query_sub_category, engine)



In [212]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 设置目标网址
url = "https://mops.twse.com.tw/mops/web/ajax_t56sb21"

trust_data = []
# year_list = ["110", "111", "112"]
year_list = ["113"]

for year in year_list:

    # 模拟表单提交的数据
    form_data = {
        "encodeURIComponent": "1",
        "run": "",
        "step": "1",
        "TYPEK": "otc",  # oct sii
        # "co_id": i,  # 公司代号
        "year": year,       # 可以填写年度，如 "2023"
        "smonth": "01",
        "emonth": "12",
        # "isnew": "false",
        "sstep": "1",
        "firstin": "true"
    }

    # 发送 POST 请求
    response = requests.post(url, data=form_data)

    # 检查请求是否成功
    if response.status_code == 200:
        # 解析返回的 HTML 内容
        soup = BeautifulSoup(response.text, 'html.parser')
        # 找到目标表格
        table = soup.find("table", {"class": "hasBorder"})

        # 初始化存储数据的列表
        # 提取表格行数据
        if table:
            rows = table.find_all("tr")[1:]  # 忽略表头
            for row in rows:
                columns = row.find_all("td")
                row_data = [col.text.strip() for col in columns]
                if(len(row_data)!=0):
                    trust_data.append(row_data)

        else:
            print("未找到相关数据表格。stock_id=")

        columns = ["異動情形", "申報日期", "公司代號", "公司名稱", "申報人身分", "姓名", "轉讓方式","轉讓股數","每日於盤中交易最大得轉讓股數","受讓人",
                "目前持有_自有持股", "目前持有_保留運用決定權信託股數","預定轉讓總股數_自有持股", "預定轉讓總股數_保留運用決定權信託股數",
                "預定轉讓後持股_自有持股", "預定轉讓後持股_保留運用決定權信託股數","有效轉讓期間","是否申報持股未完成轉讓"]


    else:
        print(f"请求失败，状态码：{response.status_code}")

    # 模拟表单提交的数据
    form_data = {
        "encodeURIComponent": "1",
        "run": "",
        "step": "1",
        "TYPEK": "sii",  # oct sii
        # "co_id": i,  # 公司代号
        "year": year,       # 可以填写年度，如 "2023"
        "smonth": "01",
        "emonth": "12",
        # "isnew": "false",
        "sstep": "1",
        "firstin": "true"
    }

    # 发送 POST 请求
    response = requests.post(url, data=form_data)

    # 检查请求是否成功
    if response.status_code == 200:
        # 解析返回的 HTML 内容
        soup = BeautifulSoup(response.text, 'html.parser')
        # 找到目标表格
        table = soup.find("table", {"class": "hasBorder"})

        # 初始化存储数据的列表
        # 提取表格行数据
        if table:
            rows = table.find_all("tr")[1:]  # 忽略表头
            for row in rows:
                columns = row.find_all("td")
                row_data = [col.text.strip() for col in columns]
                if(len(row_data)!=0):
                    trust_data.append(row_data)

        else:
            print("未找到相关数据表格。stock_id=")

        columns = ["異動情形", "申報日期", "公司代號", "公司名稱", "申報人身分", "姓名", "轉讓方式","轉讓股數","每日於盤中交易最大得轉讓股數","受讓人",
                "目前持有_自有持股", "目前持有_保留運用決定權信託股數","預定轉讓總股數_自有持股", "預定轉讓總股數_保留運用決定權信託股數",
                "預定轉讓後持股_自有持股", "預定轉讓後持股_保留運用決定權信託股數","有效轉讓期間","是否申報持股未完成轉讓"]


    else:
        print(f"请求失败，状态码：{response.status_code}")

translate_data_df = pd.DataFrame(trust_data, columns=columns)


In [213]:
# 只取信託種類來研究

trust_data_df = translate_data_df[translate_data_df["轉讓方式"] == "信託"].copy()

# 将相关列转换为数字类型（假设可能存在千位分隔符）
trust_data_df["預定轉讓總股數_自有持股"] = trust_data_df["預定轉讓總股數_自有持股"].str.replace(',', '').astype(int)
trust_data_df["目前持有_自有持股"] = trust_data_df["目前持有_自有持股"].str.replace(',', '').astype(int)
trust_data_df["轉換比例"] = trust_data_df["預定轉讓總股數_自有持股"] / trust_data_df["目前持有_自有持股"]

# 对 公司代號 进行 group by，并对 預定轉讓總股數_自有持股 进行求和
grouped_df = trust_data_df.groupby('公司代號')['預定轉讓總股數_自有持股'].sum().reset_index()

# 将结果存入新的 DataFrame
result_df = pd.DataFrame(grouped_df, columns=['公司代號', '預定轉讓總股數_自有持股'])
stock_id_list = result_df[result_df['預定轉讓總股數_自有持股'] > 450000]

# 換成看信託張數大於500張
condition_trust = trust_data_df[(trust_data_df['公司代號'].isin(stock_id_list['公司代號'])) & (trust_data_df["轉換比例"] == 1) ]

# # 最重要篩選條件
# condition_trust = trust_data_df[(trust_data_df["轉換比例"] == 1) & (trust_data_df["目前持有_自有持股"] > 100000)]  #先找100%信託仔+轉換股數不少的


# 获取 '公司代號' 和 '申報日期' 列的唯一组合
unique_combinations = condition_trust[['公司代號', '申報日期']].drop_duplicates().values.tolist()
# 打印或使用 unique_combinations 列表
unique_combinations

[['3260', '113/01/26'],
 ['4157', '113/01/17'],
 ['4163', '113/01/29'],
 ['4523', '113/07/26'],
 ['4966', '113/02/20'],
 ['5381', '113/03/18'],
 ['6576', '113/08/14'],
 ['2342', '113/07/09'],
 ['2451', '113/06/26'],
 ['2891', '113/01/12']]

In [158]:
# 用上面的新的條件
# # 只取信託種類來研究

# trust_data_df = translate_data_df[translate_data_df["轉讓方式"] == "信託"].copy()

# # 将相关列转换为数字类型（假设可能存在千位分隔符）
# trust_data_df["預定轉讓總股數_自有持股"] = trust_data_df["預定轉讓總股數_自有持股"].str.replace(',', '').astype(int)
# trust_data_df["目前持有_自有持股"] = trust_data_df["目前持有_自有持股"].str.replace(',', '').astype(int)
# trust_data_df["轉換比例"] = trust_data_df["預定轉讓總股數_自有持股"] / trust_data_df["目前持有_自有持股"]

# # 最重要篩選條件
# condition_trust = trust_data_df[(trust_data_df["轉換比例"] == 1) & (trust_data_df["目前持有_自有持股"] > 100000)]  #先找100%信託仔+轉換股數不少的
# # 获取 '公司代號' 和 '申報日期' 列的唯一组合
# unique_combinations = condition_trust[['公司代號', '申報日期']].drop_duplicates().values.tolist()
# # 打印或使用 unique_combinations 列表
# len(unique_combinations)


5

In [214]:
# filter_days = [10,30,60,90,120,150,180,210]
filter_days = [10,30,60,90,120,150,180,210]
# filter_money = 100000000
filter_money = 0

filter_day_list = pd.DataFrame(columns=['day','mean','min_mean','max_mean','count_down', 'count_up','EV'])

for filter_day in filter_days:
    stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
    # 创建一个空的 DataFrame 用于收集数据
    collected_data = pd.DataFrame(columns=['stock_id','stock_name', 'Date', '收盤價', '收盤價_min', '收盤價_max', '收盤價_mean'])


    for list in unique_combinations:
        stock_id = list[0] # stock_id8 
        roc_date = list[1] # 原始民国日期字符串

        # 1. 将民国年转换为西元年
        year, month, day = map(int, roc_date.split('/'))
        western_year = year + 1911

        # 2. 格式化日期为标准格式
        western_date_str = f"{western_year}-{month:02d}-{day:02d}"
        # 3. 转换为 datetime 对象
        western_date_o = pd.to_datetime(western_date_str)
        # 4. 加上30天
        new_date30_o = western_date_o + pd.Timedelta(days=30)
        new_date90_o = western_date_o + pd.Timedelta(days=90)
        new_date180_o = western_date_o + pd.Timedelta(days=180)

        western_date = western_date_o.strftime('%Y-%m-%d') # 信託那天

        new_date30 = new_date30_o.strftime('%Y-%m-%d') # 信託那天+30
        new_date90 = new_date90_o.strftime('%Y-%m-%d') # 信託那天+90
        new_date180 = new_date180_o.strftime('%Y-%m-%d') # 信託那天+180



        # 初始化 stock_df_temp，确保进入循环
        stock_df_temp = pd.DataFrame()
        
        # 使用 while 循环检查是否找到符合条件的记录

        # 筛选符合条件的记录
        stock_df_temp = stock_df_raw[(stock_df_raw['Date'] >= western_date) & (stock_df_raw["證券代號"] == stock_id) & (stock_df_raw['收盤價'] != 0)].copy()

        #查過去季的成值平均值
        western_date_hist = western_date_o + pd.Timedelta(days=-60)
        western_date_hist = western_date_hist.strftime('%Y-%m-%d') # 信託那天過去一季
        stock_df_hist = stock_df_raw[(stock_df_raw['Date'] >= western_date_hist) & (stock_df_raw['Date'] < western_date) & (stock_df_raw["證券代號"] == stock_id) & (stock_df_raw['收盤價'] != 0)].copy()


        stock_df_today = stock_df_temp.iloc[0] # 第一天
        stock_df_temp180 = stock_df_temp.head(filter_day)

        temp_close = stock_df_today['收盤價']
        temp_close180 = stock_df_temp180['收盤價'] 

        # 回測過去看成值沒有甚麼用 (stock_df_hist['成交金額'].mean())  但如果未來有帶量上漲 後續表現就很不錯
        # 所以只能具續觀察標的有沒有帶量上漲 可能就會有不錯的延續性
        if (stock_df_temp180['成交金額'].mean() > filter_money): # 找過去一季成交金額大於1000萬
            # 创建 DataFrame
            result_df = pd.DataFrame({
                'stock_id': [stock_id],
                'stock_name': [stock_df_today['證券名稱']],
                'Date': [stock_df_today['Date']],
                '收盤價': [stock_df_today['收盤價']],
                '收盤價_min': [temp_close180.describe()['min']],
                '收盤價_max': [temp_close180.describe()['max']],
                '收盤價_mean': [temp_close180.describe()['mean']]
            })
        else:
            continue

        # 使用 pd.concat 方法将 result_df 添加到 collected_data 中
        collected_data = pd.concat([collected_data, result_df], ignore_index=True)

    collected_data['收盤價_min%']  = round(collected_data['收盤價_min']/ collected_data['收盤價'],4)
    collected_data['收盤價_max%']  =  round(collected_data['收盤價_max']/ collected_data['收盤價'],4)
    collected_data['收盤價_mean%'] = round(collected_data['收盤價_mean']/ collected_data['收盤價'],4)

    # collected_data.sort_values(by=['收盤價_max%'], inplace=True, ascending=False)
    # collected_data.head(5)
    min_mean = collected_data['收盤價_min%'].describe()['mean']
    max_mean = collected_data['收盤價_max%'].describe()['mean']
    count_down = len(collected_data[collected_data['收盤價_mean%']<=1])
    count_up = len(collected_data[collected_data['收盤價_mean%']>1])

    EV = (min_mean*count_down+max_mean*count_up) / (count_down+count_up)


    mean_df = pd.DataFrame({
    'day': [filter_day],
    'mean': [collected_data['收盤價_mean%'].mean()],
    'min_mean':[collected_data['收盤價_min%'].describe()['mean']],
    'max_mean':[collected_data['收盤價_max%'].describe()['mean']] ,
    'count_down': [len(collected_data[collected_data['收盤價_mean%']<=1])],
    'count_up': [len(collected_data[collected_data['收盤價_mean%']>1])],
    'EV': [EV],
    })


    filter_day_list = pd.concat([filter_day_list, mean_df], ignore_index=True)



C:\Users\USER\AppData\Local\Temp\ipykernel_20612\1286665694.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  collected_data = pd.concat([collected_data, result_df], ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_20612\1286665694.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filter_day_list = pd.concat([filter_day_list, mean_df], ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_20612\1286665694.py:76: FutureWarning: The behavior of DataFrame concatenation w

In [215]:
filter_day_list

,day,mean,min_mean,max_mean,count_down,count_up,EV
0,10,0.98397,0.94080,1.03044,5,5,0.985620
1,30,0.96900,0.90484,1.04943,5,5,0.977135
2,60,0.96447,0.88565,1.08322,6,4,0.964678
3,90,0.97161,0.88333,1.10758,6,4,0.973030
4,120,0.97290,0.86044,1.11578,7,3,0.937042
5,150,0.97152,0.84238,1.11578,7,3,0.924400
6,180,0.97152,0.84238,1.11578,7,3,0.924400
7,210,0.97152,0.84238,1.11578,7,3,0.924400


In [216]:
sub_category_sub = sub_category_list[['stock_id', 'main_category']].drop_duplicates()
merged_sub = pd.merge(collected_data, sub_category_sub, on='stock_id', how='inner')  # 全部股票

collected_data.sort_values(by=['收盤價_mean%'], inplace=True, ascending=False)
collected_data.head(50)
#filter_day_list

,stock_id,stock_name,Date,收盤價,收盤價_min,收盤價_max,收盤價_mean,收盤價_min%,收盤價_max%,收盤價_mean%
9,2891,中信金,2024-01-12,28.05,27.55,40.85,33.663121,0.9822,1.4563,1.2001
0,3260,威剛,2024-01-26,94.50,82.80,120.00,102.619847,0.8762,1.2698,1.0859
1,4157,太景*-KY,2024-01-17,14.75,13.35,16.45,14.957609,0.9051,1.1153,1.0141
6,6576,逸達,2024-08-14,78.40,78.00,78.40,78.200000,0.9949,1.0000,0.9974
5,5381,合正,2024-03-18,33.90,26.45,39.15,33.737379,0.7802,1.1549,0.9952
2,4163,鐿鈦,2024-01-29,123.00,112.50,128.00,121.415385,0.9146,1.0407,0.9871
3,4523,永彰,2024-07-26,45.30,39.70,48.00,44.689286,0.8764,1.0596,0.9865
7,2342,茂矽,2024-07-09,32.70,27.95,34.70,31.800000,0.8547,1.0612,0.9725
8,2451,創見,2024-06-26,124.50,91.60,124.50,102.817143,0.7357,1.0000,0.8258
4,4966,譜瑞-KY,2024-02-20,1320.00,665.00,1320.00,858.851240,0.5038,1.0000,0.6506


count    73.000000
mean      0.859189
std       0.121945
min       0.472332
25%       0.798046
50%       0.876380
75%       0.949254
max       1.000000
Name: 收盤價_min%, dtype: float64
count    73.000000
mean      1.346908
std       0.507793
min       1.000000
25%       1.073171
50%       1.212880
75%       1.422101
max       3.963415
Name: 收盤價_max%, dtype: float64


In [166]:
stock_df_temp = stock_df_raw[(stock_df_raw['Date'] >= '2023-09-14') & (stock_df_raw["證券代號"] == '8087')].copy()
stock_df_today = stock_df_temp.iloc[0]

stock_df_temp180 = stock_df_temp.head(180)
stock_df_temp180[stock_df_temp180['收盤價']== 0]

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
1223480,2023-11-07,8087,華鎂鑫,9,1,450,0.0,0.0,0.0,0.0,...,51.3633,0.0,0.0,1.105583,0.0,1.3267,40.3235,0.0,0.0,2.874516


In [144]:
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
# 创建一个空的 DataFrame 用于收集数据
collected_data = pd.DataFrame(columns=['stock_id', 'Date', '收盤價', '收盤價_30天後', '收盤價_90天後', '收盤價_180天後'])


for list in unique_combinations:
    stock_id = list[0] # stock_id
    roc_date = list[1] # 原始民国日期字符串

    # 1. 将民国年转换为西元年
    year, month, day = map(int, roc_date.split('/'))
    western_year = year + 1911

    # 2. 格式化日期为标准格式
    western_date_str = f"{western_year}-{month:02d}-{day:02d}"
    # 3. 转换为 datetime 对象
    western_date_o = pd.to_datetime(western_date_str)
    # 4. 加上30天
    new_date30_o = western_date_o + pd.Timedelta(days=30)
    new_date90_o = western_date_o + pd.Timedelta(days=90)
    new_date180_o = western_date_o + pd.Timedelta(days=180)

    western_date = western_date_o.strftime('%Y-%m-%d') # 信託那天
    new_date30 = new_date30_o.strftime('%Y-%m-%d') # 信託那天
    new_date90 = new_date90_o.strftime('%Y-%m-%d') # 信託那天
    new_date180 = new_date180_o.strftime('%Y-%m-%d') # 信託那天



    # 初始化 stock_df_temp，确保进入循环
    stock_df_temp = pd.DataFrame()
    
    # 使用 while 循环检查是否找到符合条件的记录

    # 筛选符合条件的记录
    stock_df_temp = stock_df_raw[(stock_df_raw['Date'] >= western_date) & (stock_df_raw["證券代號"] == stock_id)].copy()
    stock_df_temp30 = stock_df_raw[(stock_df_raw['Date'] >= new_date30) & (stock_df_raw["證券代號"] == stock_id)].copy()
    stock_df_temp90 = stock_df_raw[(stock_df_raw['Date'] >= new_date90) & (stock_df_raw["證券代號"] == stock_id)].copy()
    stock_df_temp180 = stock_df_raw[(stock_df_raw['Date'] >= new_date180) & (stock_df_raw["證券代號"] == stock_id)].copy()
    
    stock_df_temp = stock_df_temp.iloc[0]
    stock_df_temp30 = stock_df_temp30.iloc[0]
    stock_df_temp90 = stock_df_temp90.iloc[0]
    stock_df_temp180 = stock_df_temp180.iloc[0]

    temp_close = stock_df_temp['收盤價']
    temp_close30 = stock_df_temp30['收盤價'] 
    temp_close90 = stock_df_temp90['收盤價'] 
    temp_close180 = stock_df_temp180['收盤價']

    # 创建 DataFrame
    result_df = pd.DataFrame({
        'stock_id': [stock_id],
        'Date': [western_date],
        '收盤價': [temp_close],
        '收盤價_30天後': [temp_close30],
        '收盤價_90天後': [temp_close90],
        '收盤價_180天後': [temp_close180]
    })

    # 使用 pd.concat 方法将 result_df 添加到 collected_data 中
    collected_data = pd.concat([collected_data, result_df], ignore_index=True)

collected_data

    

C:\Users\USER\AppData\Local\Temp\ipykernel_14604\1363887107.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  collected_data = pd.concat([collected_data, result_df], ignore_index=True)


,stock_id,Date,收盤價,收盤價_30天後,收盤價_90天後,收盤價_180天後
0,3372,2021-12-06,25.3,23.40,19.30,18.05
1,3629,2021-12-22,49.0,59.00,57.80,47.35
2,5251,2021-07-05,14.1,13.90,13.55,18.05
3,6021,2021-04-28,24.2,22.50,22.50,21.40
4,6535,2021-09-08,32.9,31.95,38.00,35.65
...,...,...,...,...,...,...
70,4915,2023-09-20,70.9,68.80,65.90,80.20
71,4938,2023-10-18,76.9,80.80,81.80,98.70
72,5388,2023-09-20,129.0,114.50,133.50,138.00
73,6625,2023-12-13,104.5,99.70,95.60,94.40


In [147]:
collected_data['收盤價_30天後%']  = collected_data['收盤價_30天後']/ collected_data['收盤價']
collected_data['收盤價_90天後%']  =  collected_data['收盤價_90天後']/ collected_data['收盤價']
collected_data['收盤價_180天後%'] = collected_data['收盤價_180天後']/ collected_data['收盤價']

collected_data.sort_values(by=['收盤價_90天後%'], inplace=True, ascending=False)


In [151]:
len(collected_data[collected_data['收盤價_30天後%']>1])

36